# Consultas Integradas

Este notebook tiene el fin de generar las consultas planteadas en el trabajo práctico integrador

#### Importo librerias

In [9]:
# Limpia todas las variables existentes
%reset -f

# Habilita autoreload para recargar automáticamente todos los módulos
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import os
from pathlib import Path
import pandas as pd
from pymongo.errors import ConnectionFailure
from db_connections import client, db_neo4j, db_redis
from src import mongo, neo4j, utils, redis
from IPython.display import display

In [11]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

✅ Conexión a MongoDB verificada.
✅ Conexión a Neo4j verificada.
✅ Conexión a Redis verificada.


### A.Mostrar los usuarios que visitaron “Bariloche”. 

In [4]:
query = """ 
MATCH (U:Usuario)-[:VISITO]->(D:Destino)
WHERE D.ciudad='La Plata'
RETURN DISTINCT U.usuario_id AS id, U.nombre AS Nombre, U.apellido AS Apellido
"""

usuarios = neo4j.consulta(db_neo4j, query)
usuarios

,id,Nombre,Apellido
0,25,Agapito,Sedano
1,45,Candela,Solano
2,53,Ovidio,Romero
3,48,Fernanda,Romero
4,42,Eliseo,Sobrino


### B.Mostrar los amigos de Juan que visitaron algún destino que visitó él, mostrar el nombre del Usuario y el destino. 

In [5]:
import pandas as pd

# Pedimos el nombre del usuario
nombre_usuario = input("Introduce tu nombre para saber qué lugares visitaste con tus amigos: ")

# Query Cypher corregida: el parámetro se llama $nombre
query = """
MATCH (u:Usuario {nombre: $nombre})-[:AMIGO_DE]-(amigo:Usuario)
MATCH (u)-[:VISITO]->(d:Destino)<-[:VISITO]-(amigo)
RETURN 
    amigo.nombre AS Nombre,
    collect(DISTINCT d.ciudad) AS Destinos_Compartidos
ORDER BY Nombre
"""

# Mensaje de encabezado
print("\n" + "-"*60)
print(f"{nombre_usuario.upper()} TUS AMIGOS VISITARON ESTOS MISMOS DESTINOS QUE TÚ")
print("-"*60 + "\n")

# Ejecutamos la consulta usando la función consulta() que devuelve un DataFrame
usuarios = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

# Mostramos el resultado
if usuarios.empty:
    print("No se encontraron amigos que hayan visitado los mismos destinos.")
else:
    print(usuarios)


Introduce tu nombre para saber qué lugares visitaste con tus amigos:  Candela



------------------------------------------------------------
CANDELA TUS AMIGOS VISITARON ESTOS MISMOS DESTINOS QUE TÚ
------------------------------------------------------------

     Nombre                               Destinos_Compartidos
0    Carmen        [San Martín de los Andes, Neuquén, Formosa]
1      Lino        [El Calafate, Posadas, Formosa, Río Cuarto]
2  Maricela                 [San Martín de los Andes, Posadas]
3    Poncio   [El Calafate, San Salvador de Jujuy, Río Cuarto]
4    Sabina                                          [Neuquén]
5  Trinidad                        [Neuquén, Posadas, Formosa]
6     Óscar  [San Martín de los Andes, San Salvador de Juju...


### C. Sugerir destinos a un usuario que no haya visitado él ni sus amigos. 

In [6]:
nombre_usuario = input("Introduce tu nombre para saber qué lugares nuevos que no visitaste ni tu ni tus amigos: ")

query = """
MATCH (d:Destino)
WHERE 
NOT EXISTS {MATCH (d)<-[:VISITO]-(u:Usuario {nombre: $nombre})}
AND 
NOT EXISTS {MATCH (d)<-[:VISITO]-(:Usuario)-[:AMIGO_DE]-(:Usuario {nombre: $nombre})}
RETURN DISTINCT d.ciudad AS Destinos_No_Visitados
ORDER BY d.ciudad
"""

print("\n" + "-"*60)
print(f"DESTINOS NUEVOS PARA {nombre_usuario.upper()} Y SUS AMIGOS")
print("-"*60 + "\n")

destinos = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

if destinos.empty:
    print("No hay destinos nuevos disponibles.")
else:
    # Mostrar cada destino en lista
    for d in destinos['Destinos_No_Visitados']:
        print(f"- {d}")



Introduce tu nombre para saber qué lugares nuevos que no visitaste ni tu ni tus amigos:  Candela



------------------------------------------------------------
DESTINOS NUEVOS PARA CANDELA Y SUS AMIGOS
------------------------------------------------------------

- La Rioja
- Santa Rosa
- Villa Carlos Paz


### d. Recomendar destinos basados en viajes de amigos.

In [7]:
nombre_usuario = input("Introduce tu nombre para saber qué lugares te recomendamos: ")

query = """
MATCH (u:Usuario {nombre:$nombre})-[:AMIGO_DE]-(amigo:Usuario)
MATCH (amigo)-[:VISITO]->(d:Destino)
WHERE NOT (u)-[:VISITO]->(d)   
RETURN DISTINCT d.ciudad AS Destino_Recomendado
ORDER BY d.ciudad
"""

print("\n" + "-"*60)
print(f"RECOMENDACIONES PARA {nombre_usuario.upper()} EN FUNCIÓN DE SUS AMIGOS")
print("-"*60 + "\n")

destinos = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

if destinos.empty:
    print("No hay destinos recomendados nuevos para vos.")
else:
    for d in destinos['Destino_Recomendado']:
        print(f"- {d}")


Introduce tu nombre para saber qué lugares te recomendamos:  Candela



------------------------------------------------------------
RECOMENDACIONES PARA CANDELA EN FUNCIÓN DE SUS AMIGOS
------------------------------------------------------------

- Bariloche
- CABA
- Chilecito
- Concordia
- Corrientes
- Córdoba
- Gualeguaychú
- Iguazú
- Mar del Plata
- Mendoza
- Merlo
- Paraná
- Puerto Madryn
- Resistencia
- Rosario
- Salta
- San Fernando
- San Juan
- San Luis
- San Miguel de Tucumán
- San Rafael
- Santa Fe
- Santiago del Estero
- Ushuaia
- Viedma


### e. Listar los hoteles en los destinos recomendados del punto anterior. 

In [8]:
nombre_base = "clase"
coleccion = "hoteles"

# Lista de destinos recomendados
lista_destinos = destinos["Destino_Recomendado"].dropna().unique().tolist()

if not lista_destinos:
    print("No hay destinos recomendados disponibles.")
else:
    filtro = {"ciudad": {"$in": lista_destinos}}
    proyeccion = {"_id": 0, "nombre": 1, "ciudad": 1, "direccion": 1}

    # Obtener datos de Mongo y convertir a DataFrame
    cursor = mongo.obtener_cursor(
        nombre_base=nombre_base,
        nombre_coleccion=coleccion,
        filtro=filtro,
        proyeccion=proyeccion
    )

    hoteles = pd.DataFrame(list(cursor))

    if hoteles.empty:
        print("No se encontraron hoteles en los destinos recomendados.")
    else:
        # Ordenar por ciudad y luego por nombre de hotel
        hoteles.sort_values(by=["ciudad", "nombre"], inplace=True)
        print(hoteles.to_string(index=False))
       



                                      nombre                ciudad
         Banca Privada Castellana S.L. Hotel             Bariloche
            Consultoría del Norte S.A. Hotel             Bariloche
               Gaya y asociados S.Com. Hotel             Bariloche
             Palmer & Asociados S.Com. Hotel             Bariloche
          Promociones Españolas S.L.U. Hotel             Bariloche
               Gonzalez y Yáñez S.Com. Hotel                  CABA
          Manuel Romeu Mayoral S.Coop. Hotel                  CABA
              Distribuciones Díez S.A. Hotel             Chilecito
         Hernando y Valenciano S.L.N.E Hotel             Chilecito
                      Hotel Serra S.L. Hotel             Chilecito
         José Mari Agustí Sola S.L.N.E Hotel             Chilecito
        Comercializadora Llorente S.L. Hotel             Concordia
                Hermanos Martín S.L.L. Hotel             Concordia
                    Nieto y Vera S.C.P Hotel             Conco

### f. Ver las reservas en proceso, es decir que aún no están concretadas. 

In [14]:
claves = db_redis.keys("reserva_temp:*")
print(f"Cantidad de reservas en proceso {len(claves)}\n")

if claves:
    cantidad = int(input ("¿Cuántas se desean listar?"))
    print(f"Se imprimen las primeras {cantidad}:")
    for clave in claves[:cantidad]:
        datos = db_redis.hgetall(clave)
        tiempo_restante = db_redis.ttl(clave)
        print(f"{clave}: {datos} | TTL: {tiempo_restante} segundos")

Cantidad de reservas temporales 0



### g. Listar los usuarios conectados actualmente. 

In [48]:
claves = db_redis.keys("usuario:*:sesion")
print(f"Cantidad de usuarios conectados {len(claves)}\n")

if claves:
    print("Usuarios:")
    for clave in claves:
        usuario_id = clave.split(":")[1]
        estado = db_redis.get(clave)
        tiempo_restante = db_redis.ttl(clave)
        print(f"Usuario {usuario_id} → sesión: {estado} | TTL: {tiempo_restante} segundos")

Cantidad de usuarios conectados 0



### h. Mostrar los destinos con precio inferior a $100.000

In [97]:
precio=100000
base = "clase"
coleccion = "destinos"
filtro={"precio_promedio": {"$lt": precio}}
proyeccion = {"_id":0}

# Busca en caché y sino consulta en MongoDB
resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    print("Consulta hecha en Mongo")
    cursor = mongo.obtener_cursor(base,coleccion,filtro=filtro,proyeccion=proyeccion)
    resultado = list(cursor)
    if resultado:
        redis.guardar_en_cache(coleccion,filtro,resultado,ttl=300)
else:
    print("Consulta hecha en Redis")
        
df = pd.DataFrame(resultado)
if not df.empty:
    df_ordenado = df.sort_values(by="destino_id")
    display(df_ordenado.style.hide(axis="index"))
else:
    print(f"No existen destinos con precio inferior a {precio}")

Consulta hecha en Mongo


destino_id,provincia,ciudad,pais,tipo,precio_promedio
1,Buenos Aires,La Plata,Argentina,Cultural,56556
3,Ciudad Autónoma de Buenos Aires,CABA,Argentina,Playa,86579
6,Chubut,Puerto Madryn,Argentina,Cultural,57811
9,Córdoba,Río Cuarto,Argentina,Relax,56956
14,Formosa,Formosa,Argentina,Cultural,91853
16,La Pampa,Santa Rosa,Argentina,Montaña,90758
18,La Rioja,Chilecito,Argentina,Cultural,74312
19,Mendoza,Mendoza,Argentina,Aventura,75353
23,Neuquén,Neuquén,Argentina,Relax,82722
24,Neuquén,San Martín de los Andes,Argentina,Aventura,70656


## i. Mostrar todos los Hoteles de “Jujuy”.

In [100]:
nombre_base = "clase"
coleccion = "hoteles"
ciudad = "San Salvador de Jujuy"
filtro = {"ciudad": ciudad}
proyeccion = {"_id": 0, "nombre": 1, "ciudad": 1, "direccion": 1}

# Busca en caché y sino consulta en MongoDB
resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    print("Consulta hecha en Mongo")
    cursor = mongo.obtener_cursor(nombre_base=nombre_base,nombre_coleccion=coleccion,filtro=filtro,proyeccion=proyeccion)
    resultado = list(cursor)
    if resultado:
        redis.guardar_en_cache(coleccion,filtro,resultado,ttl=300)
else:
    print("Consulta hecha en Redis")

df_hoteles = pd.DataFrame(resultado)

if not df_hoteles.empty:
    df_ordenado = df_hoteles.sort_values(by="nombre")
    display(df_ordenado.style.hide(axis="index"))
else:
     print("No se encontraron hoteles en los destinos recomendados.")       

nombre,ciudad
Comercializadora Soria S.L. Hotel,San Salvador de Jujuy
Familia Guitart S.L. Hotel,San Salvador de Jujuy
Hotel Botella y asociados S.L.U. Hotel,San Salvador de Jujuy
Manufacturas Puga & Asociados S.L.L. Hotel,San Salvador de Jujuy
Moliner y Torrents S.L.N.E Hotel,San Salvador de Jujuy


## j. Mostrar la cantidad de hoteles de un destino que guste. 

In [104]:
nombre_base = "clase"
coleccion = "destinos"
db = client[nombre_base]
ciudad = input ("Ingrese una ciudad que le guste")
filtro={"ciudad":ciudad}

resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    print("Consulta hecha en Mongo")
    resultado = db[coleccion].count_documents(filtro)
    if resultado:
        redis.guardar_en_cache(coleccion,filtro,resultado,ttl=300)
else:
    print("Consulta hecha en Redis")
    
print(f"La cantidad de hoteles en {ciudad} son: {resultado}") 

Ingrese una ciudad que le guste La Plata


La cantidad de hoteles en La Plata son: 1


### k. Mostrar las actividades de “Ushuaia” del tipo “aventura”.

In [105]:
nombre_base = "clase"
coleccion = "actividades"
tipo="aventura"
ciudad="Ushuaia"
filtro = {"ciudad":ciudad,"tipo":tipo}
proyeccion = {"_id":0}

resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    cursor = mongo.obtener_cursor(base,coleccion,filtro=filtro,proyeccion=proyeccion)
    print("Consulta hecha en Mongo")
    resultado = list(cursor)
    if resultado:
        redis.guardar_en_cache(coleccion,filtro,resultado,ttl=300)
else:
    print("Consulta hecha en Redis")
    
df = pd.DataFrame(resultado)
if not df.empty:
    df_ordenado = df.sort_values(by="actividad_id")
    display(df_ordenado.style.hide(axis="index"))
else:
    print(f"No existen actividades de tipo {tipo} en {ciudad}")

Consulta hecha en Mongo
No existen actividades de tipo aventura en Ushuaia


### l. Mostrar la cantidad de reservas concretadas de cada usuario. Mostrar el usuario y la cantidad 

In [112]:
nombre_base = "clase"
coleccion = "reservas"
db = client[nombre_base]
estados = ["Confirmada","Pagada"]
filtro = {"estado": {"$in":estados}}

pipeline =[ 
    {"$match": filtro},
    {"$group": {
        "_id": "$usuario_id",  # este es el ID del usuario
        "Reservas_concretadas": {"$sum": 1}
    }},
    {"$lookup": {
        "from": "usuarios",           # nombre de la colección a unir
        "localField": "_id",          # campo en esta colección (el ID del usuario)
        "foreignField": "usuario_id",        # campo en la colección "usuarios"
        "as": "usuario_info"          # nombre del nuevo campo con los datos del usuario
    }},
    {"$unwind": "$usuario_info"},     # desanida el array para acceder directamente
    {"$project": {
        "_id": 0,
        "Nombre": "$usuario_info.nombre",
        "Apellido": "$usuario_info.apellido",
        "Reservas_concretadas": 1
    }},
    {"$sort": {"Reservas_concretadas": -1}}
]

resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    print("Consulta hecha en Mongo")
    cursor = db[coleccion].aggregate(pipeline)
    resultado = list(cursor)
    if resultado:
        redis.guardar_en_cache(coleccion,filtro,resultado,ttl=300)
else:
    print("Consulta hecha en Redis")
        
df = pd.DataFrame(resultado)
df = df[["Nombre", "Apellido", "Reservas_concretadas"]]
display(df.style.hide(axis="index"))

Consulta hecha en Redis


Nombre,Apellido,Reservas_concretadas
Gracia,Naranjo,24
Carmen,Roma,22
Ovidio,Romero,21
David,Julián,20
Georgina,Ferrando,20
Mónica,Roca,20
Adrián,Belda,19
Cleto,Palomar,19
Eliseo,Sobrino,19
Gema,Yáñez,18


#### Estadísticas

###### i.Destino más visitado

###### ii.Hotel más barato

###### iii.Actividad más popular.

#### Modificaciones

###### a.Incrementar el precio de las actividades de Tucuman en 5% 

###### b. Agregar al hotel id=1 el servicio de SPA 

###### c. Eliminar el destino que desee

###### d. Eliminar un usuario que desee 

###### e. Eliminar las relaciones AMIGO_DE para un usuario que quiera. 